<a href="https://colab.research.google.com/github/magladko/NAI/blob/master/mpp4/mpp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#K-means – NAI mini-project
###*Kornel Gładkowski @s22411*

##Import libs, mount Google Drive & set project PATH

In [6]:
import matplotlib as plt
import seaborn as sns
import pandas as pd
import numpy as np

# from google.colab import drive
# drive.mount('/content/drive')

In [16]:
# PATH = "/content/drive/MyDrive/4. semestr/NAI/mpp4"

!git clone https://github.com/magladko/NAI.git
DATA_PATH = "/content/NAI/mpp4/data"

fatal: destination path 'NAI' already exists and is not an empty directory.


##Set user input

In [ ]:
#@title Enter number of class
k = 3 #@param {type:"slider", min:1, max:10, step:1}


##Read data